# TF-IDF 구하기
- 벡터화하기 전에 데이터 전처리
   - document별 형태소 분석
   - 미등록단어 처리
   - 명사만 추출
   - 불용어(stopwords) 처리
- document별 단어로 tokenize되어 있는 것을 다시 합쳐서 각각 하나의 document로 만들기
- 벡터화(vectorize) 하기
- tf-idf 값 구하기

In [5]:
import pandas as pd

In [6]:
# CSV 데이터 불러오기
clova_df = pd.read_csv("../../data/sentiment/sentiment_clova.csv", index_col=0)
kakao_df = pd.read_csv("../../data/sentiment/sentiment_mini.csv", index_col=0)
geni_df = pd.read_csv("../../data/sentiment/sentiment_gigagenie.csv", index_col=0)
nugu_df = pd.read_csv("../../data/sentiment/sentiment_nugu.csv", index_col=0)

In [7]:
# 데이터 합치기
# 긍정 document
pos_c_text = clova_df.positive_text[clova_df.positive_text.notnull()]
pos_k_text = kakao_df.positive_text[kakao_df.positive_text.notnull()]
pos_g_text = geni_df.positive_text[geni_df.positive_text.notnull()]
pos_n_text = nugu_df.positive_text[nugu_df.positive_text.notnull()]

In [8]:
# 데이터 합치기
# 부정 document
neg_c_text = clova_df.negative_text[clova_df.negative_text.notnull()]
neg_k_text = kakao_df.negative_text[kakao_df.negative_text.notnull()]
neg_g_text = geni_df.negative_text[geni_df.negative_text.notnull()]
neg_n_text = nugu_df.negative_text[nugu_df.negative_text.notnull()]

In [9]:
clova_text = [pos_c_text, neg_c_text]
kakao_text = [pos_k_text, neg_k_text]
geni_text = [pos_g_text, neg_g_text]
nugu_text = [pos_n_text, neg_n_text]

In [10]:
len(clova_text), len(kakao_text), len(geni_text), len(nugu_text)

(2, 2, 2, 2)

In [42]:
clova_text[1]

3        유선랜을 연결하지 않고 와이파이 연결만으로 사용이 가능하다. 그러나 음성인식 스피...
5        이 가운데 국내 업체들만 세계 시장에서 맥을 못추고 있다는 지적이 나오고 있다. ...
6                                하지만 아직 갈 길이 멀다는 지적도 나온다.
12       글을 쓰면서도 가끔은 이거 사기치는 건 아닌가? 아주 기본적인 사례입니다. 네이버...
16       여긴 어디야라는 질문도 마찬가지로 대답하지 못했다. 다만 네이버음악듣기 결제를 하...
18       올해 안으로 가입자 만명을 모으는 게 목표다. 구글홈은 검색엔진 구글과 연동돼 정...
19                           인공지능이 학습하는 데이터가 계속 쌓이기 때문이다.
31       앱은 서비스별로 따로 존재하는데 말만으로도 특정 기능을 실현시킬 수 있는 단계가 ...
32       네이버는 현재 진행 중인 차 시범 판매 행사가 끝나도 당분간 국내 정식판매 계획은...
39                                        이같은 일이 처음은 아니다.
41       구글 홈은 북미에서 달러만여원에 판매 중이다.두 회사 간 경쟁은 현지 제휴 시장에...
44       카카오톡은 왜 해외 진출 하지 않느냐는 질문을 자주 받는다. 전 안된다고 본다. ...
45       구글 어시스턴트 한국어 서비스는 이날 국내 판매가 시작된 LG전자 스마트폰 V을 ...
50       IT공룡들이 앞다퉈 경쟁하는 걸 보니 궁금해졌다. 써봐야 알지. 웨이브를 찾고 웨...
51                                       비공개 테스트후 안드로이드 .
52       구글은 구글 홈 출시 일정이 확정되지 않았지만 출시 가능성을 열어두고 있다.장 매...
53                  어렵고 복잡한 명령어를 내리면 여지없이 도와드릴 수 없다고 말한다.
54            

In [ ]:
'blog.naver.com/ppang7942/'로 시작하는 url 페이지의 데이터 거르기

# <br>
## 클로바

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [11]:
from konlpy.tag import Komoran

In [12]:
komoran = Komoran(userdic='./unregistered.txt')

In [20]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

In [22]:
results = []
for i, brand_text in enumerate(clova_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

pos tagging - 0
noun extract - 0
finish - 0
pos tagging - 1
noun extract - 1
finish - 1


### pickle로 일단 저장

In [23]:
import pickle
pickle.dump(results, open('tfidf_clova_list.pickle', "wb"))

In [25]:
load_result = pickle.load(open('tfidf_clova_list.pickle', "rb"))
results = load_result

<br/><br/>
### stopword 제거

In [26]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

In [27]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    print("finish -", i)


finish - 0



finish - 1


<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [28]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [29]:
documents[0][:200]

'직장인 김 모 미국 출장 당시 인공지능 AI 스피커 구글 홈 사 아내 아들 영어 공부 부탁 김 영어 유치원 아들 구글 홈 대화 학부모 조언 스피커 정보 확인 검색 음악 알람 아이 흥미 영어 사용 대학 강모 네이버 스피커 웨이브 이벤트 선물 내년 교환 학생 계획 강 웨이브 영어 회화 기능 활용 거란 강 네이버 번역 파파 활용 기본 회화 습득 활용 웨이브 대화'

In [44]:
documents[1]

'유선 래 연결 와이파이 연결 사용 음성 인식 스피커 일상 차지 역할 흐름 가운데 국내 업체 세계 시장 맥 지적 전문가 국내 업체 시장 확대 이유 길 지적 글 사기 기본 사례 네이버 카카오 날 하루 수준 카메라 인식 로그인 질문 마찬가지 대답 네이버 음악 듣기 결제 상태 지원 별도 이퀄라이저 설정 지하철역 자동차 추천 경로 검색 비교 효과 결과 근처 분식집 검색 질문 검색 결과 웨이브 탑재 리 마인 기능 질문 답 결론 시점 웨이브 자비스 한참 수준 가입자 게 목표 구글 홈 검색 엔진 구글 연동 정보 검색 판매량 공개 미국 시장 점유 중반 파악 인공지능 학습 데이터 서비스 존재 특정 기능 실현 단계 서비스 필요 생태계 변화 적응 번성 도태 종이 마련 네이버 진행 시범 판매 행사 국내 정식 판매 계획 입장 처음 구글 홈 북미 달러 여원 판매 회사 경쟁 현지 제휴 시장 격화 조짐 카카오톡 해외 진출 질문 국가 세컨드 메신저 의미 국가 메신저 전국 민이 플랫폼 게 국가 플랫폼 의미 계좌 개설 이모티콘 다운 설치 게 유통 파워 곳도 존재 국내 업체 카카오 네이버 린지 전만 모바일 메신저 사업 경쟁자 사업자 무한 경쟁 데이터 비용 말씀 거지 플랫폼 중립 플랫폼 중립 동일 선상 카카오 네이버 대표 교체 설 이야기 날짜 기업 생물 대화 대화 게 CEO 주년 소회 회사 성과 게 성과 길 업데이트 소프트웨어 업데이트 하드웨어 공급 준비 게 카메라 게 포털 카카오 시너지 내부 포털 사이트 매각 게 언급 사이트 포털 카카오 식 이름 논의 게 매각 논의 플랫폼 액션 필요 끝 액션 완결 카카오 베스트 카카오 브레인 투자 구조 당장 연결 계획 투자 논의 국회 고용 노동부 연결 권리 이야기 퇴근 연결 권리 의미 아 젠 다라 기술 활용 현재 시점 베타 타사 번역 서비스 번역 베타서비스 출시 모바일 웹 페이지 A B 번역 게 발전 부분 Q 기계 룰 베이스 A 질문 B 답 주시 노력 제지 이슈 게 논의 답 카카오 하루하루 생활 실무 하루하루 회의 이슈 해결 Q 수익 모델 시각 임 대표 시행착오

<br/><br/>
### 벡터화 하기

In [30]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [68]:
cv=CountVectorizer(max_df=0.60, max_features=20000, stop_words=stopwords)
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

/Users/Taewoo/anaconda2/envs/python36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_flash_removecallback', 'gyste7gb', 'kbs', 'mbc뉴스', 'news1', 'sbs', 'sbs화면캡처무기수', '무단', '전재'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [69]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

20000

In [70]:
cv.vocabulary_

{'출장': 16705,
 '학부모': 18761,
 '습득': 10969,
 '조선일보': 15579,
 'naver': 3130,
 '왓슨': 12701,
 'cc': 789,
 '에이브': 11909,
 '고의': 6102,
 '인공신경망': 14202,
 '스파크': 10930,
 '김진형': 6877,
 '정보기술': 15273,
 'airi': 146,
 '원장': 13113,
 '김범수': 6801,
 '어벤져스': 11752,
 '코리': 17091,
 '아선': 11368,
 '민규': 8880,
 '점프': 15210,
 'ua': 5064,
 '캡틴': 16979,
 '아메리카': 11354,
 '헐크': 19304,
 '토르': 17606,
 '팬서': 18065,
 '닥터': 7365,
 '스트레인지': 10916,
 '눈앞': 7253,
 '여종': 12056,
 '원혁': 13140,
 'aiot': 139,
 '히어로': 19984,
 '특장': 17824,
 '수요처': 10729,
 'tvskt': 5016,
 '예정지': 12405,
 '라운드': 7984,
 '레노': 8043,
 '확산': 19695,
 '이통': 14132,
 '성장세': 10385,
 '광화문': 6328,
 '사옥': 9898,
 '설명회': 10333,
 'vod': 5330,
 '단장': 7391,
 '최준': 16617,
 '상무': 10065,
 '종로구': 15705,
 '놀이학습': 7212,
 '연계': 12104,
 '박명순': 8986,
 '유닛': 13315,
 '협업': 19440,
 '막바지': 8346,
 '촉진': 16575,
 '야구': 11610,
 '골프': 6146,
 '오다': 12434,
 '연례': 12133,
 '대회': 7560,
 'io': 2044,
 '네스': 7118,
 '감탄': 5695,
 '윤곽': 13504,
 '조리법': 15551,
 '탐지': 17464,
 '손짓': 10607,
 '개막':

<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [71]:
komoran.pos("번개장터")

[('번개장터', 'NNG')]

In [72]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [73]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

배포                   0.27243329240656544
전재                   0.2429411903869102
오디오북                 0.1961224784307075
협업                   0.16441846875957808
연계                   0.15815139708040132
led                  0.14598590499729355
강조                   0.12460413103304348
파트너                  0.11686245425288397
vod                  0.11575650042714689
저작권                  0.10469696216977617
ces                  0.10248505451830203
자회사                  0.10137910069256495
ybm                  0.09732393666486236
장터                   0.0954806802886339
전자신문                 0.09400607518765114
번개                   0.0921628188114227
알리미                  0.09179416753617699
lte                  0.08515844458175456
개방                   0.08442114203126318
개최                   0.08331518820552611

timestamp            0.24214029368137843
조곤                   0.24214029368137843
datetime             0.20178357806781538
plc                  0.20178357806781538
황치열                

# <br>
## 카카오

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [74]:
from konlpy.tag import Komoran

In [75]:
komoran = Komoran(userdic='./unregistered.txt')

In [76]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

In [77]:
results = []
for i, brand_text in enumerate(kakao_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

pos tagging - 0
noun extract - 0
finish - 0
pos tagging - 1
noun extract - 1
finish - 1


### pickle로 일단 저장

In [78]:
import pickle
pickle.dump(results, open('tfidf_kakao_list.pickle', "wb"))

In [79]:
load_result2 = pickle.load(open('tfidf_kakao_list.pickle', "rb"))
results = load_result2

<br/><br/>
### stopword 제거

In [80]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

In [81]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    #print("finish -", i)

<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [82]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [83]:
documents[0][:200]

'이해인 카카오 미니 확인 전송 가능 네이버 웨이브 음악 서비스 당 결제 카카오 미니 출시 추석 연휴 예고 국내 인터넷 서비스 투 톱 AI 인공지능 스피커 경쟁 카카오 카카오 미니 네이버 웨이브 예약 판매 분도 채 준비 물량 소진 관심 몸 상황 웨이브 네이버 기술력 기반 음성 인식 출력 스피커 장점 카카오 미니 국내 최적화 서비스 무장 기본 제품 음성 명령 '

<br/><br/>
### 벡터화 하기

In [157]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [84]:
cv=CountVectorizer(max_df=0.85, max_features=20000, stop_words=stopwords)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

/Users/Taewoo/anaconda2/envs/python36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_flash_removecallback', 'gyste7gb', 'kbs', 'mbc뉴스', 'news1', 'sbs', 'sbs화면캡처무기수', '무단'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [85]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

20000

In [86]:
cv.vocabulary_

{'이해인': 14595,
 '예고': 13110,
 '무장': 9235,
 '외출': 13445,
 '사용성': 10256,
 '물리': 9289,
 '고사': 6904,
 '만화': 8943,
 '치어리더': 17271,
 '레이싱': 8657,
 '헤르츠': 19448,
 '전략적': 15421,
 '음폭': 14165,
 '현장감': 19524,
 '음역': 14159,
 '프로세서': 18698,
 '고음': 6924,
 '감시': 6572,
 '손상': 10962,
 '극장': 7356,
 '이기범': 14262,
 '블로터': 10087,
 '아카데미': 12164,
 '역량': 12873,
 '진화': 16590,
 '사이버': 10268,
 '귀사': 7307,
 'cbs': 566,
 '노컷뉴스': 7854,
 '김수영': 7565,
 '네모': 7799,
 '상당수': 10413,
 'goodhertz': 1749,
 'tm': 3974,
 '하만카돈': 18889,
 'ch': 608,
 '레인지': 8664,
 '드라이버': 8469,
 '증강': 16333,
 '시브': 11610,
 '에이': 12712,
 'passive': 2953,
 'radiator': 3246,
 '유닛': 13916,
 'general': 1666,
 '어도비': 12539,
 'adobe': 58,
 '응답자': 14175,
 '재미': 15256,
 '주식회사': 16132,
 '커넥트': 17449,
 '관광': 7066,
 '명소': 9104,
 '플러그인': 18744,
 '에이전시': 12723,
 '의뢰': 14193,
 '여자친구': 12854,
 '광화문': 7131,
 '걷기': 6715,
 '상권': 10408,
 '단발': 8037,
 '복수': 9864,
 '관이': 7086,
 '입장료': 14915,
 '윤정희': 14093,
 '만족도': 8935,
 '디지털타임스': 8534,
 '연중': 12966,
 '풀뿌리': 18625

<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [88]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

lg유플러스               0.19084881455971134
생태계                  0.16639331319653225
편의                   0.14193781183335316
클라우드                 0.13810165475677605
기아                   0.13138837987276608
진화                   0.12131846754675119
연구소                  0.1179618301047462
시너지                  0.10549431960587058
비즈니스                 0.10165816252929348
시각                   0.10117864289472134
오디오북                 0.09974008399100491
설계                   0.09878104472186064
가전제품                 0.09638344654899995
조원                   0.09494488764528354
카카오i                 0.09158825020327856
보일러                  0.08583401458841289
자녀                   0.08535449495384075
향상                   0.08535449495384075
iptv                 0.08343641641555219
패턴                   0.08247737714640792

개각                   0.21110016546037466
ssid                 0.1266600992762248
그늘                   0.1266600992762248
낙엽                   0.1266600992762248
다이어터               